In [1]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
spec = cv2.imread("specs/class0/1.png", 0)
#plt.imshow(spec, cmap="gray")
spec.max()

242

In [3]:
images = []
labels = []

for i in range(10):
    path = "specs/class" + str(i)
    num = len(os.listdir(path))
    for j in range(num):
        img_path = path + "/" + str(j) + ".png"
        img = cv2.imread(img_path, 0)
        img = img / 255
        images.append(img)
        labels.append(i)

images = np.array(images)
labels = np.array(labels)

print(f"len(images) = {len(images)}")
print(f"len(labels) = {len(labels)}")

len(images) = 8732
len(labels) = 8732


In [4]:
images.shape

(8732, 100, 100)

In [5]:
from sklearn.model_selection import train_test_split

train_size = 0.75
x_train, x_test, y_train, y_test = train_test_split(images, labels, train_size=train_size)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

#x_test, x_dev, y_test, y_dev = train_test_split(x_test, y_test)

x_train = x_train.reshape(int(len(images)*train_size), 100, 100, 1)
x_test = x_test.reshape(int(len(images)*(1-train_size)), 100, 100, 1)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
#plt.imshow(x_train[0], cmap="gray")

(6549, 100, 100) (6549,) (2183, 100, 100) (2183,)
(6549, 100, 100, 1) (6549,) (2183, 100, 100, 1) (2183,)


In [6]:
from keras.utils import to_categorical

y_cat_train = to_categorical(y_train, 10)
print(y_cat_train.shape, y_cat_train[0])

y_cat_test = to_categorical(y_test, 10)

(6549, 10) [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [13]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.losses import categorical_crossentropy
import warnings
warnings.filterwarnings("ignore")

In [25]:
# Let's compare 4 different architectures, from simple to more complex.

img_shape = (100, 100, 1)
model1 = Sequential([
    Conv2D(filters=32, kernel_size=(4, 4), input_shape=img_shape, activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=32, kernel_size=(4, 4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model1.compile(optimizer="adam", loss=categorical_crossentropy, metrics=['accuracy'])

model2 = Sequential([
    Conv2D(filters=64, kernel_size=(4, 4), input_shape=img_shape, activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(4, 4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])

model2.compile(optimizer="adam", loss=categorical_crossentropy, metrics=['accuracy'])

model3 = Sequential([
    Conv2D(filters=64, kernel_size=(4, 4), input_shape=img_shape, activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(4, 4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model3.compile(optimizer="adam", loss=categorical_crossentropy, metrics=['accuracy'])

model4 = Sequential([
    Conv2D(filters=64, kernel_size=(4, 4), input_shape=img_shape, activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(4, 4), input_shape=img_shape, activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(4, 4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    #QUESTIN: is adding the following line better? Or when does it become better?
    #Dropout(rate=0.5),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model4.compile(optimizer="adam", loss=categorical_crossentropy, metrics=['accuracy'])


In [26]:
model1.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 97, 97, 32)        544       
                                                                 
 max_pooling2d_36 (MaxPooli  (None, 48, 48, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_37 (Conv2D)          (None, 45, 45, 32)        16416     
                                                                 
 max_pooling2d_37 (MaxPooli  (None, 22, 22, 32)        0         
 ng2D)                                                           
                                                                 
 flatten_16 (Flatten)        (None, 15488)             0         
                                                                 
 dense_40 (Dense)            (None, 128)             

In [27]:
model2.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_38 (Conv2D)          (None, 97, 97, 64)        1088      
                                                                 
 max_pooling2d_38 (MaxPooli  (None, 48, 48, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_39 (Conv2D)          (None, 45, 45, 64)        65600     
                                                                 
 max_pooling2d_39 (MaxPooli  (None, 22, 22, 64)        0         
 ng2D)                                                           
                                                                 
 flatten_17 (Flatten)        (None, 30976)             0         
                                                                 
 dense_42 (Dense)            (None, 256)             

In [28]:
model3.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 97, 97, 64)        1088      
                                                                 
 max_pooling2d_40 (MaxPooli  (None, 48, 48, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_41 (Conv2D)          (None, 45, 45, 64)        65600     
                                                                 
 max_pooling2d_41 (MaxPooli  (None, 22, 22, 64)        0         
 ng2D)                                                           
                                                                 
 flatten_18 (Flatten)        (None, 30976)             0         
                                                                 
 dense_44 (Dense)            (None, 256)             

In [23]:
model4.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_33 (Conv2D)          (None, 97, 97, 64)        1088      
                                                                 
 max_pooling2d_33 (MaxPooli  (None, 48, 48, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_34 (Conv2D)          (None, 45, 45, 64)        65600     
                                                                 
 max_pooling2d_34 (MaxPooli  (None, 22, 22, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_35 (Conv2D)          (None, 19, 19, 64)        65600     
                                                                 
 max_pooling2d_35 (MaxPooli  (None, 9, 9, 64)        

In [38]:
print("TRAINING MODEL 1\n")
model1.fit(x_train, y_cat_train, epochs=5, verbose=1)

print("TRAINING MODEL 2\n")
model2.fit(x_train, y_cat_train, epochs=5, verbose=1)

print("TRAINING MODEL 3\n")
model3.fit(x_train, y_cat_train, epochs=5, verbose=1)

print("TRAINING MODEL 4\n")
model4.fit(x_train, y_cat_train, epochs=5, verbose=1)

TRAINING MODEL 1

Epoch 1/5
205/205 [==============================] - 7s 32ms/step - loss: 0.1037 - accuracy: 0.9696
Epoch 2/5
205/205 [==============================] - 7s 32ms/step - loss: 0.0705 - accuracy: 0.9803
Epoch 3/5
205/205 [==============================] - 6s 31ms/step - loss: 0.0572 - accuracy: 0.9841
Epoch 4/5
205/205 [==============================] - 6s 31ms/step - loss: 0.0377 - accuracy: 0.9901
Epoch 5/5
205/205 [==============================] - 6s 31ms/step - loss: 0.0439 - accuracy: 0.9878
TRAINING MODEL 2

Epoch 1/5
205/205 [==============================] - 20s 99ms/step - loss: 0.0674 - accuracy: 0.9792
Epoch 2/5
205/205 [==============================] - 20s 100ms/step - loss: 0.0659 - accuracy: 0.9818
Epoch 3/5
205/205 [==============================] - 20s 100ms/step - loss: 0.0881 - accuracy: 0.9750
Epoch 4/5
205/205 [==============================] - 20s 100ms/step - loss: 0.0384 - accuracy: 0.9896
Epoch 5/5
205/205 [==============================] - 20s 

In [39]:
models = [model1, model2, model3, model4]

In [43]:
# SONUÇLAR ÜZERİNE
#   Yukarıdaki 4 modeli 10 (5*2) epochla çalıştırdım. 
#   İsabetlilik oranları 5 epochta %84 düzeylerindeydi.
#   Daha büyük epochlarla denenebilir.
#   Daha da büyük katmanlar için "Dropout"lar eklenebilir.

for model in models:
    model.evaluate(x_test, y_cat_test)

69/69 [==============================] - 2s 25ms/step - loss: 0.5828 - accuracy: 0.8681


In [33]:
predictions = model1.predict(x_test)
predictions

69/69 [==============================] - 1s 8ms/step


array([[6.0110739e-09, 7.8043610e-10, 9.8648397e-03, ..., 7.3012588e-06,
        2.2697767e-02, 5.2296491e-05],
       [1.0533574e-03, 5.9860533e-05, 1.1721845e-04, ..., 7.0630005e-05,
        9.7982893e-06, 9.9267112e-04],
       [2.5021469e-09, 2.4967584e-10, 1.6446811e-11, ..., 9.9983907e-01,
        8.5550299e-12, 1.2019110e-09],
       ...,
       [7.6263185e-10, 8.9066324e-11, 9.7994572e-01, ..., 6.7931062e-08,
        1.9754166e-02, 2.7037340e-05],
       [2.4441193e-05, 6.0214067e-04, 2.5293099e-05, ..., 9.9569595e-01,
        5.9601472e-04, 2.4638046e-04],
       [9.7692639e-01, 2.7796562e-04, 3.2611460e-05, ..., 8.8773621e-03,
        2.4134095e-04, 1.2571682e-02]], dtype=float32)

In [34]:
print(predictions[0], predictions[0].max())
print(predictions.shape)
#Her sınıf için ihtimaller veriyor. argmax yeter.

[6.0110739e-09 7.8043610e-10 9.8648397e-03 9.6735340e-01 1.9837647e-05
 4.5854372e-06 2.1205171e-11 7.3012588e-06 2.2697767e-02 5.2296491e-05] 0.9673534
(2183, 10)


In [35]:
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes

array([3, 4, 7, ..., 2, 7, 0], dtype=int64)

In [36]:
y_test

array([3, 4, 7, ..., 2, 7, 0])

In [37]:
correct = np.sum(predicted_classes == y_test)
print(f"Number of true classifications was {correct}.\nThis makes our accuracy in test {correct/len(y_test)}")

# Neden manuel? NEDEN OLMASIN?

Number of true classifications was 1857.
This makes our accuracy in test 0.8506642235455795
